# Deepまで潜りたい小説自動生成

# 0. そもそも文章の自動生成とは 
### 文章を「単語のリスト」と捉えて、単語の順番を学習し、入力の単語から次に来そうな単語を次々と予測することで、尤もらしい文章を生成すること

### 例えば：
#### 〜〜〜に入る言葉を考えてみる（なんでもいい）
- 今日**〜〜〜**
- 目覚まし**〜〜〜**
- バナナ**〜〜〜**
---
#### 回答例：
- 今日 -> **の天気は晴れ**
- 目覚まし -> **が鳴らなくて寝坊した**
- バナナ -> **といったら黄色**
---
### パターンを知っているだけ文章を作れる
- 今日 -> **の天気は晴れ**
- 今日 -> **の天気は曇り**
- 今日 -> **の天気は雨**
- 今日 -> **はなんだか疲れた**
- 今日 -> **から夏休みでうれしい**

### これを機械にやらせるのが今日のハンズオン!!

# 1. 形態素解析
### 文章を単語に分ける技術。仕組み的には辞書マッチング。単語の切れ目がわかりにくい言語（日本語等）の前処理で必須

### 機械は文章を「単語のリスト」と捉えている
- 今日から夏休みでうれしい
  - ["今日", "から", "夏休み", "で", "うれしい"]

In [1]:
"""形態素解析してみよう①
- MeCab
- Yahoo形態素解析
等が有名
"""
import requests

sentence = "今日から夏休みでうれしい"
response = requests.get('https://mecab-server-dot-cyberagent-105.appspot.com/parse?q={}'.format(sentence)).json()
response['result']

[{'features': ['名詞', '副詞可能', '*', '*', '*', '*', '今日', 'キョウ', 'キョー'],
  'surface': '今日'},
 {'features': ['助詞', '格助詞', '一般', '*', '*', '*', 'から', 'カラ', 'カラ'],
  'surface': 'から'},
 {'features': ['名詞', '一般', '*', '*', '*', '*', '夏休み', 'ナツヤスミ', 'ナツヤスミ'],
  'surface': '夏休み'},
 {'features': ['助詞', '格助詞', '一般', '*', '*', '*', 'で', 'デ', 'デ'],
  'surface': 'で'},
 {'features': ['形容詞', '自立', '*', '*', '形容詞・イ段', '基本形', 'うれしい', 'ウレシイ', 'ウレシイ'],
  'surface': 'うれしい'}]

In [2]:
"""形態素解析してみよう②
"""
[chunk['features'] for chunk in response['result']]

[['名詞', '副詞可能', '*', '*', '*', '*', '今日', 'キョウ', 'キョー'],
 ['助詞', '格助詞', '一般', '*', '*', '*', 'から', 'カラ', 'カラ'],
 ['名詞', '一般', '*', '*', '*', '*', '夏休み', 'ナツヤスミ', 'ナツヤスミ'],
 ['助詞', '格助詞', '一般', '*', '*', '*', 'で', 'デ', 'デ'],
 ['形容詞', '自立', '*', '*', '形容詞・イ段', '基本形', 'うれしい', 'ウレシイ', 'ウレシイ']]

In [3]:
"""形態素解析してみよう③
"""
[chunk['features'][6] for chunk in response['result']]

['今日', 'から', '夏休み', 'で', 'うれしい']

In [4]:
"""形態素解析してみよう④
"""
def split_chunks(sentence):
    response = requests.get('https://mecab-server-dot-cyberagent-105.appspot.com/parse?q={}'.format(sentence)).json()
    return [chunk['features'][6] for chunk in response['result']]

split_chunks("今日から夏休みでうれしい")

['今日', 'から', '夏休み', 'で', 'うれしい']

# 2. マルコフ連鎖
### 最初の単語の、次の単語の候補をいくつか覚えていて、その中から１つ選んで繋げていく方法

In [5]:
"""まずは文章を学習させる
試しに青空文庫の小説を覚えさせる
http://www.aozora.gr.jp/index.html
"""

!wget http://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip
!unzip 789_ruby_5639.zip
!rm -f 789_ruby_5639.zip
!head wagahaiwa_nekodearu.txt

--2017-09-01 04:34:46--  http://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip
Resolving www.aozora.gr.jp (www.aozora.gr.jp)... 59.106.13.115
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350404 (342K) [application/zip]
Saving to: ‘789_ruby_5639.zip’

789_ruby_5639.zip   100%[===================>] 342.19K  --.-KB/s    in 0.1s    

2017-09-01 04:34:46 (2.54 MB/s) - ‘789_ruby_5639.zip’ saved [350404/350404]

Archive:  789_ruby_5639.zip
  inflating: wagahaiwa_nekodearu.txt  
��y�͔L�ł���
�Ėڟ���

-------------------------------------------------------
�y�e�L�X�g���Ɍ����L���ɂ��āz

�s�t�F���r
�i��j��y�s�킪�͂��t�͔L�ł���

�b�F���r�̕t��������̎n�܂����肷��L��


In [6]:
"""文字コードがつらいのでUTF-8に変換...
"""

import codecs
with codecs.open('wagahaiwa_nekodearu.txt', 'r', 'shift_jis') as r:
    with codecs.open('wagahaiwa_nekodearu_utf8.txt', 'w', 'utf-8') as w:
        w.write(r.read())

!rm -f wagahaiwa_nekodearu.txt
!head -26 wagahaiwa_nekodearu_utf8.txt

In [7]:
"""先頭と最後の余計な文の掃除とルビの削除と改行の調整
"""

!sed -i '1,24d' wagahaiwa_nekodearu_utf8.txt
!sed -i '2336,$d' wagahaiwa_nekodearu_utf8.txt
!sed -i -E 's/《[^》]+》//g' wagahaiwa_nekodearu_utf8.txt
!cat wagahaiwa_nekodearu_utf8.txt | tr -d '\n' | tr -d '\r' | sed -E 's/。/。\n/g' > wagahaiwa_nekodearu_parsed.txt
!head -10 wagahaiwa_nekodearu_parsed.txt

　吾輩は猫である。
名前はまだ無い。
　どこで生れたかとんと見当がつかぬ。
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
吾輩はここで始めて人間というものを見た。
しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ。
この書生というのは時々我々を捕えて煮て食うという話である。
しかしその当時は何という考もなかったから別段恐しいとも思わなかった。
ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。
掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。


In [8]:
"""前処理終わり、単語の繋がりを作ってみる①

['今日', 'から', '夏休み', 'で', 'うれしい']

- bigram
  - ２文字ずつずらしてグループ化して繋げる
  - 「今日から」「から夏休み」「夏休みで」「でうれしい」
- trigram
  - 3文字ずつずらしてグループ化して繋げる
  - 「今日から夏休み」「から夏休みで」「夏休みでうれしい」
- n-gram
  - n文字ずつグループ化して繋げる
"""

chunks = split_chunks("今日から夏休みでうれしい。")

# 2文字の組み合わせ（bigram）をカウント。よく出てくる組み合わせは生成のときに優先度上げたりできる
bigram_freqs = {}
for i in range(len(chunks) - 1):
    bigram = tuple(chunks[i : i+2])  # bigramを作る（i文字目からi+2文字目まで）
    if bigram in bigram_freqs:
        bigram_freqs[bigram] += 1
    else:
        bigram_freqs[bigram] = 1

bigram_freqs

{('うれしい', '。'): 1,
 ('から', '夏休み'): 1,
 ('で', 'うれしい'): 1,
 ('今日', 'から'): 1,
 ('夏休み', 'で'): 1}

In [9]:
"""単語の繋がりを作ってみる②
"""

# 文頭と文末はフラグをつけてカウント
bigram_freqs[('__BEGIN__', chunks[0], chunks[1])] = 1
bigram_freqs[(chunks[-2], chunks[-1], '__END__')] = 1

bigram_freqs

{('__BEGIN__', '今日', 'から'): 1,
 ('うれしい', '。'): 1,
 ('うれしい', '。', '__END__'): 1,
 ('から', '夏休み'): 1,
 ('で', 'うれしい'): 1,
 ('今日', 'から'): 1,
 ('夏休み', 'で'): 1}

In [10]:
"""単語の繋がりを作ってみる③
ここまでをメソッドにしてみる
"""

def make_bigram(chunks):
    # 1文字以下なら組み合わせが作れないのでreturn
    if len(chunks) < 2:
        return {}
    
    # 2文字の組み合わせ（bigram）をカウント。よく出てくる組み合わせは生成のときに優先度上げたりできる
    bigram_freqs = {}
    for i in range(len(chunks) - 1):
        bigram = tuple(chunks[i : i+2])  # bigramを作る（i文字目からi+2文字目まで）
        if bigram in bigram_freqs:
            bigram_freqs[bigram] += 1
        else:
            bigram_freqs[bigram] = 1

    # 文頭と文末はフラグをつけてカウント
    bigram_freqs[('__BEGIN__', chunks[0], chunks[1])] = 1
    bigram_freqs[(chunks[-2], chunks[-1], '__END__')] = 1

    return bigram_freqs

chunks = split_chunks("今日から夏休みでうれしい。")
make_bigram(chunks)

{('__BEGIN__', '今日', 'から'): 1,
 ('うれしい', '。'): 1,
 ('うれしい', '。', '__END__'): 1,
 ('から', '夏休み'): 1,
 ('で', 'うれしい'): 1,
 ('今日', 'から'): 1,
 ('夏休み', 'で'): 1}

In [191]:
# %%time
"""文章全部でbigramを集計①
小説に出てくる単語の繋がりを覚えさせる処理

...ローカルでやると20分くらいかかるので用意したものをロードしよう
"""

# # 実行しないこと
# bigram_freqs_all = {}
# sentences = open('wagahaiwa_nekodearu_parsed.txt').read().replace('\u3000', '').split('\n')
# for sentence in sentences:
#     chunks = split_chunks(sentence)
#     bigrams = make_bigram(chunks)
#     for (bigram, n) in bigrams.items():
#         if bigram in bigram_freqs_all:
#             bigram_freqs_all[bigram] += n
#         else:
#             bigram_freqs_all[bigram] = 1

# bigram_freqs_all

In [11]:
import pandas as pd

"""文章全部でbigramを集計②
集計済のbigramをロードしてみる
"""

!wget https://storage.googleapis.com/chck/handson/170901text_generation/bigram_dict_waganeko.pkl
dict_path = './bigram_dict_waganeko.pkl'
# pd.DataFrame(list(bigram_freqs_all.items()), columns=['bigram', 'freq']).to_pickle(dict_path)
pd.read_pickle(dict_path)

--2017-09-01 04:35:09--  https://storage.googleapis.com/chck/handson/170901text_generation/bigram_dict_waganeko.pkl
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.25.208, 2404:6800:4004:81a::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.25.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1385066 (1.3M) [application/octet-stream]
Saving to: ‘bigram_dict_waganeko.pkl’

bigram_dict_waganek 100%[===================>]   1.32M  --.-KB/s    in 0.1s    

2017-09-01 04:35:09 (9.21 MB/s) - ‘bigram_dict_waganeko.pkl’ saved [1385066/1385066]



,bigram,freq
0,"(吾輩は猫である, 。)",2
1,"(__BEGIN__, 吾輩は猫である, 。)",1
2,"(吾輩は猫である, 。, __END__)",2
3,"(名前, は)",7
4,"(は, まだ)",20
5,"(まだ, 無い)",2
6,"(無い, 。)",3
7,"(__BEGIN__, 名前, は)",5
8,"(無い, 。, __END__)",3
9,"(どこ, で)",5


In [12]:
"""作ったbigram辞書で単語を検索してみる①
"""

bigram_dict = pd.read_pickle(dict_path)
bigram_dict[bigram_dict.bigram.apply(lambda row: row[0] == 'ニャーニャー')]

,bigram,freq
28,"(ニャーニャー, 泣く)",1
18062,"(ニャーニャー, と)",3


In [13]:
"""作ったbigram辞書で単語を検索してみる②
メソッド化してみる
"""

def find_bigrams(chunk):
    bigram_dict = pd.read_pickle(dict_path)
    return bigram_dict[bigram_dict.bigram.apply(lambda row: row[0] == chunk)]

find_bigrams('ニャーニャー')

,bigram,freq
28,"(ニャーニャー, 泣く)",1
18062,"(ニャーニャー, と)",3


In [14]:
"""ついに文の生成部分へ①
ひっかかった組み合わせの中から適当に１つ選ぶメソッドをつくる
"""
import random

def find_bigram_atramdom(chunk):
    bigrams = find_bigrams(chunk)
    return random.choice(bigrams.values)[0]

find_bigram_atramdom('__BEGIN__')

('__BEGIN__', 'いくら', '駄弁')

In [19]:
"""ついに文の生成部分へ②
"""
chunks = []
first_bigram = find_bigram_atramdom('__BEGIN__')

# __BEGIN__を省くために[1]から
chunks.append(first_bigram[1])
chunks.append(first_bigram[2])
chunks

['但し', '檜']

In [20]:
"""ついに文の生成部分へ③
繋げすぎて迷子になって__END__までたどり着けない場合があるのでたまに失敗します
"""

# 一番最後の単語が__END__になるまで繰り返しbigramを繋げていく
while chunks[-1] != '__END__':
    bigram = find_bigram_atramdom(chunks[-1]) # 　文の一番後ろの単語でbigramを検索して繋げる
    chunks.append(bigram[1]) # bigram[0] は検索単語と一緒なので繋げる部分はbigram[1]だけ
    if len(bigram) == 3:
        chunks.append(bigram[-1])  # if条件がイケてないがbigramのサイズが3==__END__なので__END__を繋げておわり
        
''.join(chunks[:-1]) # 最後の__END__を消してリスト->文字列にして完成

'但し檜がかつてここにおいて望の…あなた。'

In [21]:
"""ついに文の生成部分へ④
ここまでをメソッド化
"""

def generate_sentence():
    chunks = []
    first_bigram = find_bigram_atramdom('__BEGIN__')

    # __BEGIN__を省くために[1]から
    chunks.append(first_bigram[1])
    chunks.append(first_bigram[2])
    
    # 一番最後の単語が__END__になるまで繰り返しbigramを繋げていく
    while chunks[-1] != '__END__':
        bigram = find_bigram_atramdom(chunks[-1]) # 　文の一番後ろの単語でbigramを検索して繋げる
        chunks.append(bigram[1]) # bigram[0] は検索単語と一緒なので繋げる部分はbigram[1]だけ
        if len(bigram) == 3:
            chunks.append(bigram[-1])  # if条件がイケてないがbigramのサイズが3==__END__なので__END__を繋げておわり

    return  ''.join(chunks[:-1]) # 最後の__END__を消してリスト->文字列にして完成

generate_sentence()

'足懸るの葉を頂戴」彼がすこぶる奇観の痘痕面らあ体に較べるてはおるんことを見廻す。'

In [24]:
"""ついに文の生成部分へ⑤
完全ランダムで次の単語を選ぶのはイケてないので、よく出てくる単語を優先して繋げるようにする
"""

def find_bigram_probable(chunk):
    bigrams = find_bigrams(chunk) # 指定単語から始まる組み合わせを辞書から全検索
    
    candidates = [] # 出現回数でいい感じにした組み合わせ候補
    for idx, row in bigrams.iterrows():
        for _ in range(row.freq):
            """freq==出現回数
            freqが高いほど次に選ばれる確率を高くするために、出現回数分、候補のリストに入れちゃう
            """
            candidates.append(idx)
            
    # 候補の中から１つ選ぶ
    return bigrams.ix[random.choice(candidates)].bigram

In [31]:
"""ついに文の生成部分へ⑥
完全ランダムで次の単語を選ぶのはイケてないので、よく出てくる単語を優先して繋げるようにする
"""

def generate_sentence_v2():
    chunks = []
    first_bigram = find_bigram_probable('__BEGIN__')

    # __BEGIN__を省くために[1]から
    chunks.append(first_bigram[1])
    chunks.append(first_bigram[2])
    
    # 一番最後の単語が__END__になるまで繰り返しbigramを繋げていく
    while chunks[-1] != '__END__':
        bigram = find_bigram_probable(chunks[-1]) # 　文の一番後ろの単語でbigramを検索して繋げる
        chunks.append(bigram[1]) # bigram[0] は検索単語と一緒なので繋げる部分はbigram[1]だけ
        if len(bigram) == 3:
            chunks.append(bigram[-1])  # if条件がイケてないがbigramのサイズが3==__END__なので__END__を繋げておわり

    return  ''.join(chunks[:-1]) # 最後の__END__を消してリスト->文字列にして完成

generate_sentence_v2()

'四時に目覚しい食*。'

### v2のほうがちょっと改善してるはず！！
### おしまい

# 3. LSTM
### 文を先頭からちょっとずつ読んでいって、重要そうな単語だけメモしつつ要約していく方法
### DeepLearningの1手法

In [138]:
"""RNN
"""

'RNN\n'

In [ ]:
"""LSTM
"""

In [ ]:
"""Seq2Seq
"""

In [ ]:
"""まとめ
自然言語処理は前処理9割の学問です
"""